# Zero to Hero

## 1.12 Funcion de Ganancia de Arbol training/testing

El objetivo es poner el notebook anterior dentro de una funcion, e invocarla con distintas semillas, para maravillarnos ante la dispersion de las ganancias

In [ ]:
rm(list = ls())
gc()

library("data.table") # cargo la libreria  data.table
library("rpart") # cargo la libreria  rpart
library("rpart.plot")
library("caret")

options(repr.plot.width = 20, repr.plot.height = 10)
setwd("~/buckets/b1/")

In [ ]:
GananciaArbol <- function(semilla, data, x, train = 0.70) {
  # establezco la semilla
  set.seed(semilla)
  train_rows <- createDataPartition(dataset$clase_ternaria, p = 0.70, list = FALSE)

  modelo <- rpart::rpart(
    formula = "clase_ternaria ~ .",
    data = data[train_rows], # los datos de training
    xval = 0,
    control = x
  )

  # Aplico el modelo a los datos de testing
  prediccion <- predict(modelo, data[-train_rows], type = "prob")

  prob_baja2 <- prediccion[, "BAJA+2"]
  ganancia_testing <- data[-train_rows, sum((prob_baja2 > 0.025) * ifelse(clase_ternaria == "BAJA+2", 117000, -3000))]

  return(ganancia_testing)
}


Vale la pena notar que dentro de la funcion no se han creado  dtrain y dtest,  sino que directamente se utilizan:
* data[ train_rows]  como training
* data[ -train_rows]  como testing


#Aqui empieza el programa

In [ ]:
dataset <- fread("~/datasets/vivencial_dataset_pequeno.csv") # cargo el dataset #cargo el dataset
dataset <- dataset[foto_mes == 202107] # me quedo solo con el periodo 202107


In [ ]:
# defino unos buenos hiperparametros
param <- list(
    "cp" = -0.5,
    "minsplit" = 500,
    "minbucket" = 10,
    "maxdepth" = 5
)


In [ ]:
# Ahora hago algunas llamadas a la funcion


In [ ]:
GananciaArbol(11, dataset, x = param)


In [ ]:
GananciaArbol(13, dataset, x = param)


In [ ]:
GananciaArbol(17, dataset, x = param)


In [ ]:
GananciaArbol(19, dataset, x = param)


In [ ]:
GananciaArbol(23, dataset, x = param)


Lo primero que se obseva es la variabilidad de la ganancia sin normalizar segun la semilla, hasta un 10%

Esta dispersión es **NOTABLE**  ya que el algoritmo es el mismo, con los mismos parámetros.
<br>
Lo único que cambia es que datos se utilizan para entrenar y testear, pero siempre es 70% / 30%
<br>
Es más notable aún que se ha tenido cuidado que la partición sea estratificada segun el campo **clase_ternaria**, lo que apriori uno supondria que va a generar particiones muy homogéneas.